Example 1 - Calculate Equivalence Ratio
https://cantera.org/stable/examples/python/thermo/equivalenceRatio.html#sphx-glr-examples-python-thermo-equivalenceratio-py

In [0]:
import cantera as ct

gas = ct.Solution('gri30.yaml')
air = "O2:0.21,N2:0.79"

gas.set_equivalence_ratio(phi=1.0, fuel="CH4:1", oxidizer=air)
phi = gas.equivalence_ratio()
print(f"phi = {phi:1.3f}")

Z = gas.mixture_fraction(fuel="CH4:1", oxidizer=air)
print(f"Bilger Mixture Fraction, Z = {Z:1.3f}")

print(f"mass fraction of CH4 = {gas['CH4'].Y[0]:1.3f}")

Example 2 - Extracting a sub-mechanism
https://cantera.org/stable/examples/python/kinetics/extract_submechanism.html#sphx-glr-examples-python-kinetics-extract-submechanism-py

In [0]:
from timeit import default_timer
import matplotlib.pyplot as plt

input_file = 'gri30.yaml'
all_species = ct.Species.list_from_file(input_file)
species = []

for S in all_species:
    comp = S.composition
    if 'C' in comp and 'H' in comp:
        # Exclude all hydrocarbon species
        continue
    if 'N' in comp and comp != {'N': 2}:
        # Exclude all nitrogen compounds except for N2
        continue

    species.append(S)

species_names = {S.name for S in species}
print('Species: {0}'.format(', '.join(S.name for S in species)))

ref_phase = ct.Solution(thermo='ideal-gas', kinetics='gas', species=all_species)
all_reactions = ct.Reaction.list_from_file(input_file, ref_phase)
reactions = []

print('\nReactions:')
for R in all_reactions:
    if not all(reactant in species_names for reactant in R.reactants):
        continue

    if not all(product in species_names for product in R.products):
        continue

    reactions.append(R)
    print(R.equation)
print('\n')

gas1 = ct.Solution(input_file)
gas2 = ct.Solution(name="gri30-CO-H2-submech",
                   thermo="ideal-gas", kinetics="gas",
                   transport_model="mixture-averaged",
                   species=species, reactions=reactions)

gas2.update_user_header({"description": "CO-H2 submechanism extracted from GRI-Mech 3.0"})
gas2.write_yaml("gri30-CO-H2-submech.yaml", header=True)

Example 3 - Using new mechanism for flame simulations 
https://cantera.org/stable/examples/python/onedim/burner_flame.html 

In [0]:
from pathlib import Path

p = 0.05 * ct.one_atm
tburner = 373.0
mdot = 0.06
reactants = 'H2:1.5, O2:1, AR:7'  # premixed gas composition
width = 0.5  # m
loglevel = 1  # amount of diagnostic output (0 to 5)

gas = ct.Solution('gri30-CO-H2-submech.yaml')
gas.TPX = tburner, p, reactants

f = ct.BurnerFlame(gas, width=width)
f.burner.mdot = mdot
f.set_refine_criteria(ratio=3.0, slope=0.05, curve=0.1)
f.show()

f.transport_model = 'mixture-averaged'
f.solve(loglevel, auto=True)

if "native" in ct.hdf_support():
    output = Path() / "burner_flame.h5"
else:
    output = Path() / "burner_flame.yaml"
output.unlink(missing_ok=True)

f.save(output, name="mix", description="solution with mixture-averaged transport")

f.transport_model = 'multicomponent'
f.solve(loglevel)  # don't use 'auto' on subsequent solves
f.show()
f.save(output, name="multi", description="solution with multicomponent transport")

f.save('burner_flame.csv', basis="mole", overwrite=True)

Example 4 - Laminar Flame Speed Calculation 
https://cantera.org/stable/examples/python/onedim/adiabatic_flame.html

In [0]:
import numpy as np


# Simulation parameters
p = ct.one_atm  # pressure [Pa]
Tin = 300.0  # unburned gas temperature [K]
reactants = 'H2:1.1, O2:1, AR:5'  # premixed gas composition
width = 0.03  # m
loglevel = 1  # amount of diagnostic output (0 to 8)

# Solution object used to compute mixture properties, set to the state of the
# upstream fuel-air mixture
gas = ct.Solution('h2o2.yaml')
gas.TPX = Tin, p, reactants

# Set up flame object
f = ct.FreeFlame(gas, width=width)
f.set_refine_criteria(ratio=3, slope=0.06, curve=0.12)
f.show()

# Solve with mixture-averaged transport model
f.transport_model = 'mixture-averaged'
# Compute diffusive fluxes using a mass fraction-based gradient ("mass")
# or mole fraction-based gradient ("mole", default)
f.flux_gradient_basis = "mass" # only relevant for mixture-averaged model
f.solve(loglevel=loglevel, auto=True)

f.show()
print(f"mixture-averaged flamespeed = {f.velocity[0]:7f} m/s")

# Solve with mixture-averaged transport model and Soret diffusion
f.soret_enabled = True
f.transport_model = 'multicomponent'
f.solve(loglevel=loglevel) # don't use 'auto' on subsequent solves

f.show()
print("mixture-averaged flamespeed with Soret diffusion"
      f" = {f.velocity[0]:7f} m/s")

if "native" in ct.hdf_support():
    output = Path() / "adiabatic_flame.h5"
else:
    output = Path() / "adiabatic_flame.yaml"
output.unlink(missing_ok=True)

f.save(output, name="mix", description="solution with mixture-averaged "
                                       "transport and Soret diffusion")

# Solve with multi-component transport properties
# but without Soret diffusion
f.transport_model = 'multicomponent'
f.soret_enabled = False
f.solve(loglevel)
f.show()
print("multicomponent flamespeed without Soret diffusion"
      f" = {f.velocity[0]:7f} m/s")

# Solve with multi-component transport properties and Soret diffusion
f.soret_enabled = True
f.solve(loglevel)
f.show()
print("multicomponent flamespeed with Soret diffusion"
      f" = {f.velocity[0]:7f} m/s")

f.save(output, name="multi", description="solution with multicomponent transport "
                                         "and Soret diffusion")

# write the velocity, temperature, density, and mole fractions to a CSV file
f.save('adiabatic_flame.csv', basis="mole", overwrite=True)

Example 5 - Plasma Reactor 
 https://cantera.org/stable/examples/python/reactors/plasma.html

In [0]:
import numpy as np
import scipy.integrate

class ReactorOde:
    def __init__(self, plasma):
        # Parameters of the ODE system and auxiliary data are stored in the
        # ReactorOde object.
        self.gas = plasma
        self.P = plasma.P
        self.T = plasma.T

    def __call__(self, t, y):
        """the ODE function, y' = f(t,y) """

        # State vector is [Y_1, Y_2, ... Y_K]
        self.gas.set_unnormalized_mass_fractions(y)
        self.gas.TP = self.T, self.P
        rho = self.gas.density

        wdot = self.gas.net_production_rates
        dYdt = wdot * self.gas.molecular_weights / rho

        return np.hstack(dYdt)


plasma = ct.Solution('example_data/oxygen-plasma-itikawa.yaml',
                     'isotropic-electron-energy-plasma',
                      transport_model=None)

P = ct.one_atm * 0.01
T = 300.0
plasma.TPX = T, P, 'O2:1.0, e:5e-3, O2+:5e-3'
plasma.mean_electron_energy = 10 # [eV]
y0 = np.hstack(plasma.Y)

# Set up objects representing the ODE and the solver
ode = ReactorOde(plasma)
solver = scipy.integrate.ode(ode)
solver.set_integrator('vode', method='bdf', with_jacobian=True)
solver.set_initial_value(y0, 0.0)

# Integrate the equations, keeping T(t) and Y(k,t)
t_end = 1e-6
states = ct.SolutionArray(plasma, 1, extra={'t': [0.0]})
dt = 1e-9
while solver.successful() and solver.t < t_end:
    solver.integrate(solver.t + dt)
    plasma.TPY = T, P, solver.y
    states.append(plasma.state, t=solver.t)

# Plot the results
import matplotlib.pyplot as plt
plt.plot(states.t, states('e').X, label='e', lw=2)
plt.ylabel('Electron Mole Fraction')
plt.xlabel('Time [s]')
plt.tight_layout()
plt.savefig("plasma-density")
plt.show()

Example 6 - Continuously Stirred Reactor 

https://cantera.org/stable/examples/python/reactors/periodic_cstr.html



In [0]:
gas = ct.Solution('gri30-CO-H2-submech.yaml')

# pressure = 60 Torr, T = 770 K
p = 60.0*133.3
t = 770.0

gas.TPX = t, p, 'H2:2, O2:1'

upstream = ct.Reservoir(gas)

cstr = ct.IdealGasReactor(gas)
cstr.volume = 10.0*1.0e-6

env = ct.Reservoir(gas)
w = ct.Wall(cstr, env, A=1.0, U=0.02)

sccm = 1.25
vdot = sccm * 1.0e-6 / 60.0 * ((ct.one_atm / gas.P) * (gas.T / 273.15))  # m^3/s
mdot = gas.density * vdot  # kg/s
mfc = ct.MassFlowController(upstream, cstr, mdot=mdot)

downstream = ct.Reservoir(gas)
v = ct.Valve(cstr, downstream, K=1.0e-9)

network = ct.ReactorNet([cstr])
t = 0.0
dt = 0.1

states = ct.SolutionArray(gas, extra=['t'])
while t < 300.0:
    t += dt
    network.advance(t)
    states.append(cstr.thermo.state, t=t)

aliases = {'H2': 'H$_2$', 'O2': 'O$_2$', 'H2O': 'H$_2$O'}
for name, alias in aliases.items():
    gas.add_species_alias(name, alias)

fig, ax = plt.subplots()
for spc in aliases.values():
    ax.plot(states.t, states(spc).Y, label=spc)
plt.legend(loc='upper right')
plt.xlabel('time [s]')
plt.ylabel('mass fraction')
plt.show()